# Collisions 

# TODO
'case_id' #OK
'collision_date': str, #OK,
'collision_severity': #OK
'collision_time': str, #OK, 
'county_city_location': str, #OK 
'hit_and_run': #OK
'jurisdiction': #OK
'lighting': #OK
'location_type': #OK
'officer_id': #OK
'pcf_violation': #OK
'pcf_violation_category': #OK,
'pcf_violation_subsection': #OK,
'population': #OK
'primary_collision_factor': #OK,
'process_date': str, #OK,
'ramp_intersection': #OK
'road_condition_1': #OK,
'road_condition_2': #OK,
'road_surface': #OK,
'tow_away': #OK,
'type_of_collision': #OK,
'weather_1': #OK,
'weather_2': #OK,

In [61]:
import pandas as pd
import numpy as np
import datetime
import time

In [62]:
dtype_collisions={
    'case_id': str, # cannot use in tsince biggest case id is 9.870011e+18, np.uint64: 64-bit unsigned integer (from 0 to 2**64-1)
    'collision_date': str, #datetime.date,
    'collision_severity': str,# => convert to ID
    'collision_time': str, #time, 
    'county_city_location': str,
    'hit_and_run': str,# => convert to ID
    'jurisdiction': pd.Int16Dtype(),
    'lighting': str,
    'location_type': str,
    'officer_id': str,
    'pcf_violation': pd.Int64Dtype(),
    'pcf_violation_category': str,
    'pcf_violation_subsection': str,
    'population': pd.Int64Dtype(),
    'primary_collision_factor': str,
    'process_date': str, #datetime.date,
    'ramp_intersection': pd.Int64Dtype(),
    'road_condition_1': str,
    'road_condition_2': str,
    'road_surface': str,
    'tow_away': 'boolean',
    'type_of_collision': str,
    'weather_1': str,
    'weather_2': str, 
}

In [63]:
collisions_data = pd.read_csv("CSV-2018/collisions2018.csv", dtype=dtype_collisions ,usecols=dtype_collisions.keys()) #

In [64]:
collisions_data

,case_id,collision_date,collision_severity,collision_time,county_city_location,hit_and_run,jurisdiction,lighting,location_type,officer_id,...,primary_collision_factor,process_date,ramp_intersection,road_condition_1,road_condition_2,road_surface,tow_away,type_of_collision,weather_1,weather_2
0,0000001,2002-01-18,property damage only,15:30:00,1900,not hit and run,9575,daylight,NaN,16418,...,vehicle code violation,2002-06-14,<NA>,normal,NaN,dry,True,rear end,clear,NaN
1,0000002,2002-02-13,property damage only,19:30:00,1500,misdemeanor,9420,NaN,NaN,14360,...,vehicle code violation,2002-03-29,<NA>,normal,NaN,dry,False,hit object,clear,NaN
2,0000003,2002-02-11,property damage only,15:30:00,1502,misdemeanor,9420,daylight,highway,16117,...,vehicle code violation,2003-07-15,<NA>,normal,NaN,dry,False,hit object,clear,NaN
3,0000004,2002-02-12,property damage only,07:45:00,1500,misdemeanor,9420,daylight,NaN,6078,...,vehicle code violation,2002-05-22,<NA>,normal,NaN,dry,False,sideswipe,clear,NaN
4,0000005,2002-02-14,severe injury,11:35:00,1500,not hit and run,9420,daylight,NaN,13851,...,vehicle code violation,2003-07-28,<NA>,normal,NaN,dry,True,overturned,clear,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678058,9870011224092016011,2001-12-24,pain,09:20:00,3600,not hit and run,9870,daylight,highway,16011,...,vehicle code violation,2002-04-26,7,normal,NaN,dry,True,hit object,clear,NaN
3678059,9870011224123011850,2001-12-24,property damage only,12:30:00,3600,not hit and run,9870,daylight,intersection,11850,...,vehicle code violation,2002-04-26,5,normal,NaN,dry,False,sideswipe,clear,NaN
3678060,9870011226102009803,2001-12-26,other injury,10:20:00,3600,not hit and run,9870,daylight,NaN,9803,...,vehicle code violation,2002-03-29,8,normal,NaN,dry,True,head-on,clear,NaN
3678061,9870011228210011458,2001-12-28,property damage only,21:00:00,3600,misdemeanor,9870,dark with no street lights,NaN,11458,...,vehicle code violation,2002-03-08,8,normal,NaN,dry,False,hit object,cloudy,NaN


# Data Cleaning

Translation tables

In [65]:
translation_collision_severity ={
    "fatal": 1,
    "severe injury": 2,
    "other injury": 3,
    "pain": 4,
    "property damage only": 0,
}

translation_hit_and_run ={
    "felony": "F",
    "misdemeanor": "M",
    "not hit and run": "N",
}

translation_lighting ={
    "daylight": "A",
    "dusk or dawn": "B",
    "dark with street lights": "C",
    "dark with no street lights": "D",
    "dark with street lights not functioning": "E",
}

translation_location_type ={
    "highway": "H",
    "intersection": "I",
    "ramp": "R",
}

translation_pcf_violation_category ={
    'unknown': 0,
    'dui': 1,
    'impeding traffic': 2,
    'speeding': 3,
    'following too closely': 4,
    'wrong side of road': 5,
    'improper passing': 6,
    'unsafe lane change': 7,
    'improper turning': 8,
    'automobile right of way':9,
    'pedestrian right of way': 10,
    'pedestrian violation': 11,
    'traffic signals and signs': 12,
    'hazardous parking': 13,
    'lights': 14,
    'brakes': 15,
    'other equipment': 16,
    'other hazardous violation': 17,
    'other than driver (or pedestrian)': 18,
    'unsafe starting or backing': 19,
    'other improper driving': 22,
    'pedestrian dui': 23,
    'fell asleep': 24,    
}

translation_primary_collision_factor ={
    "vehicle code violation": "A",
    "other improper driving": "B",
    "other than driver": "C",
    "unknown": "D",
    "fell asleep": "E",
}

translation_road_condition ={
    'holes': "A",
    'loose material': "B",
    'obstruction': "C",
    'construction': "D",
    'reduced width':"E",
    'flooded': "F",
    'other':"G",
    'normal': "H",
}

translation_road_surface ={
    'dry': "A",
    'wet': "B",
    'snowy': "C", 
    'slippery': "D", 
}

translation_type_of_collision ={
    'head-on': "A",
    'sideswipe':"B",
    'rear end':"C",
    'broadside':"D",
    'hit object':"E",
    'overturned':"F",
    'pedestrian':"G",
    'other':"H",
}

translation_weather ={
    'clear': "A", 
    'cloudy': "B",
    'raining': "C",
    'snowing':"D",
    'fog': "E",
    'other': "F", 
    'wind':"G",
}

Clean columns that need to be cleaned

In [66]:
clean_data["collision_severity"] = collisions_data["collision_severity"].map(lambda s: translation_collision_severity[s]).astype(pd.Int16Dtype())
clean_data["hit_and_run"] = collisions_data["hit_and_run"].map(lambda s: translation_hit_and_run.get(s, None))
clean_data["lighting"] = collisions_data["lighting"].map(lambda s: translation_lighting.get(s, None))
clean_data["location_type"] = collisions_data["location_type"].map(lambda s: translation_location_type.get(s, None))
clean_data["primary_collision_factor"] = collisions_data["primary_collision_factor"].map(lambda s: translation_primary_collision_factor.get(s, None))
clean_data["pcf_violation_category"] = collisions_data["pcf_violation_category"].map(lambda s: translation_pcf_violation_category.get(s, None)).astype(pd.Int16Dtype())
clean_data["road_condition_1"] = collisions_data["road_condition_1"].map(lambda s: translation_road_condition.get(s, None))
clean_data["road_condition_2"] = collisions_data["road_condition_2"].map(lambda s: translation_road_condition.get(s, None))
clean_data["road_surface"] = collisions_data["road_surface"].map(lambda s: translation_road_surface.get(s, None))
clean_data["type_of_collision"] = collisions_data["type_of_collision"].map(lambda s: translation_type_of_collision.get(s, None))
clean_data["weather_1"] = collisions_data["weather_1"].map(lambda s: translation_weather.get(s, None))
clean_data["weather_2"] = collisions_data["weather_2"].map(lambda s: translation_weather.get(s, None))
clean_data["process_date"] = collisions_data["process_date"].astype("datetime64[ns]").dt.date
clean_data["datetime"] = (collisions_data["collision_date"] + " "+ collisions_data["collision_time"]).astype("datetime64[ns]")

In [67]:
set(clean_data["collision_severity"])

{0, 1, 2, 3, 4}

In [68]:
clean_data

,case_id,collision_date,collision_severity,collision_time,county_city_location,hit_and_run,jurisdiction,lighting,location_type,officer_id,...,process_date,ramp_intersection,road_condition_1,road_condition_2,road_surface,tow_away,type_of_collision,weather_1,weather_2,datetime
0,0000001,2002-01-18,0,15:30:00,1900,N,9575,A,None,16418,...,2002-06-14,<NA>,H,None,A,True,C,A,None,2002-01-18 15:30:00
1,0000002,2002-02-13,0,19:30:00,1500,M,9420,None,None,14360,...,2002-03-29,<NA>,H,None,A,False,E,A,None,2002-02-13 19:30:00
2,0000003,2002-02-11,0,15:30:00,1502,M,9420,A,H,16117,...,2003-07-15,<NA>,H,None,A,False,E,A,None,2002-02-11 15:30:00
3,0000004,2002-02-12,0,07:45:00,1500,M,9420,A,None,6078,...,2002-05-22,<NA>,H,None,A,False,B,A,None,2002-02-12 07:45:00
4,0000005,2002-02-14,2,11:35:00,1500,N,9420,A,None,13851,...,2003-07-28,<NA>,H,None,A,True,F,A,None,2002-02-14 11:35:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3678058,9870011224092016011,2001-12-24,4,09:20:00,3600,N,9870,A,H,16011,...,2002-04-26,7,H,None,A,True,E,A,None,2001-12-24 09:20:00
3678059,9870011224123011850,2001-12-24,0,12:30:00,3600,N,9870,A,I,11850,...,2002-04-26,5,H,None,A,False,B,A,None,2001-12-24 12:30:00
3678060,9870011226102009803,2001-12-26,3,10:20:00,3600,N,9870,A,None,9803,...,2002-03-29,8,H,None,A,True,A,A,None,2001-12-26 10:20:00
3678061,9870011228210011458,2001-12-28,0,21:00:00,3600,M,9870,D,None,11458,...,2002-03-08,8,H,None,A,False,E,B,None,2001-12-28 21:00:00


                                ------------------------------------------

# Create CSV files

## Collisions CSV

In [69]:
collisions_csv = clean_data.copy()

collision_rename_translate = {"collision_severity": "collision_severity_id", "hit_and_run": "hit_and_run_id", "lighting": "lighting_id", "location_type": "location_type_id", "pcf_violation_category": "pcf_violation_category_id", "population":"population_id", "primary_collision_factor": "primary_collision_factor_id", "ramp_intersection":"ramp_intersection_id", "road_surface":"road_surface_id", "type_of_collision": "type_of_collision_id"}
collision_drop = ["road_condition_1", "road_condition_2", "weather_1", "weather_2", "collision_date", "collision_time"]

collisions_csv.rename(columns=collision_rename_translate, inplace=True)
collisions_csv.drop(collision_drop , axis="columns",  inplace=True)

In [70]:
collisions_csv.to_csv('out/collision.csv', index=False)

In [85]:
#collisions_csv.loc[[1314887]]

for x in collisions_csv["case_id"]:
    if("017477" in x):
        print(x)

collisions_csv[collisions_csv["case_id"] == "0017477"]
    

0017477
0174771
0174772
0174773
0174774
0174775
0174776
0174777
0174778
1017477
3017477


,case_id,collision_severity_id,county_city_location,hit_and_run_id,jurisdiction,lighting_id,location_type_id,officer_id,pcf_violation,pcf_violation_category_id,pcf_violation_subsection,population_id,primary_collision_factor_id,process_date,ramp_intersection_id,road_surface_id,tow_away,type_of_collision_id,datetime
16003,0017477,0,1942,M,1942,B,None,35786,22350,3,NaN,7,A,2002-06-24,<NA>,A,False,C,2002-02-28 17:45:00


## with_weather relation CVS

In [9]:
translate= {"weather_1": "wheather_id", "weather_2": "wheather_id"}
weather_1 = clean_data[~clean_data["weather_1"].isnull()][["case_id", "weather_1"]].copy().rename(columns=translate)
weather_2 = clean_data[~clean_data["weather_2"].isnull()][["case_id", "weather_2"]].copy().rename(columns=translate)
with_weather_csv = pd.concat([weather_1, weather_2], axis=0)

In [10]:
with_weather_csv.to_csv('out/collision_with_weather.csv', index=False)

## with_road_condition relation CVS

In [11]:
translate= {"road_condition_1": "road_condition_id", "road_condition_2": "road_condition_id"}
road_condition_1 = clean_data[~clean_data["road_condition_1"].isnull()][["case_id", "road_condition_1"]].copy().rename(columns=translate)
road_condition_2 = clean_data[~clean_data["road_condition_2"].isnull()][["case_id", "road_condition_2"]].copy().rename(columns=translate)
with_road_condition_csv = pd.concat([road_condition_1, road_condition_2], axis=0)

In [12]:
with_road_condition_csv.to_csv('out/collision_with_road_condition.csv', index=False)